In [12]:
#영화 추천 알고리즘
#사용자의 이전 영화시청기록이나 평점을 기반으로 추천하는 방법
import numpy as np

# 가상의 사용자-영화 평점 행렬을 생성합니다.
# 사용자A부터 D까지 4명, 영화1부터 5까지 5편의 영화로 가정합니다.
# 각 셀은 해당 사용자가 해당 영화에 대해 매긴 평점을 나타냅니다. (0은 평점 미부여를 의미)
ratings = np.array([
    [5, 4, 0, 0, 1],
    [0, 0, 5, 4, 2],
    [4, 5, 1, 2, 0],
    [1, 2, 4, 5, 0]
])

# 사용자 간 유사도를 계산하는 함수 (코사인 유사도 사용)
def cosine_similarity(user1, user2):
    return np.dot(user1, user2) / (np.linalg.norm(user1) * np.linalg.norm(user2))

# 사용자별로 다른 사용자와의 유사도 계산해서 매트릭스로 나타냄
def compute_similarity_matrix(ratings):
    num_users = ratings.shape[0]
    similarity_matrix = np.zeros((num_users, num_users))
    for i in range(num_users):
        for j in range(num_users):
            similarity_matrix[i, j] = cosine_similarity(ratings[i], ratings[j])
    return similarity_matrix

# 사용자에게 영화를 추천하는 함수
def get_movie_recommendations(user_id, ratings, similarity_matrix, num_recommendations=3):
    user_ratings = ratings[user_id]
    similar_users = np.argsort(similarity_matrix[user_id])[::-1]  # 유사도 높은 사용자 순으로 정렬
    recommendations = []
    for movie_id in range(ratings.shape[1]):
        if user_ratings[movie_id] == 0:  # 해당 영화를 아직 평가하지 않은 경우에만 추천
            weighted_sum = 0
            weight_sum = 0
            for similar_user in similar_users:
                if ratings[similar_user, movie_id] > 0:  # 해당 영화에 대해 평가한 경우에만 사용
                    similarity = similarity_matrix[user_id, similar_user]
                    weighted_sum += similarity * ratings[similar_user, movie_id]
                    weight_sum += similarity
            if weight_sum > 0:
                predicted_rating = weighted_sum / weight_sum
                recommendations.append((movie_id, predicted_rating))
    recommendations.sort(key=lambda x: x[1], reverse=True)
    return recommendations[:num_recommendations]

similarity_matrix = compute_similarity_matrix(ratings)

# 결과 출력
for user_id in range(ratings.shape[0]):
    recommended_movies = get_movie_recommendations(user_id, ratings, similarity_matrix)
    print(f"사용자(인덱스 {user_id+1})에게 영화 추천:")
    for movie_id, rating in recommended_movies:
        print(f"영화 {movie_id + 1}: 예상 평점 {rating:.2f}")


사용자(인덱스 1)에게 영화 추천:
영화 4: 예상 평점 2.78
영화 3: 예상 평점 1.86
사용자(인덱스 2)에게 영화 추천:
영화 2: 예상 평점 2.78
영화 1: 예상 평점 1.86
사용자(인덱스 3)에게 영화 추천:
영화 5: 예상 평점 1.24
사용자(인덱스 4)에게 영화 추천:
영화 5: 예상 평점 1.75


In [16]:
from transformers import pipeline

# 가상의 영화 데이터: 영화와 관련된 특성 정보
movies_data = {
    "The Dark Knight": {"genre": "action", "cast": ["Christian Bale", "Heath Ledger"], "director": "Christopher Nolan"},
    "La La Land": {"genre": "romance", "cast": ["Ryan Gosling", "Emma Stone"], "director": "Damien Chazelle"},
    "Deadpool": {"genre": "comedy", "cast": ["Ryan Reynolds"], "director": "Tim Miller"},
    "The Conjuring": {"genre": "horror", "cast": ["Vera Farmiga", "Patrick Wilson"], "director": "James Wan"},
    # 추가 영화 정보
}

# 감정 분석을 위한 Sentiment Analysis 모델 로드
sentiment_analysis_model = pipeline("sentiment-analysis")

# 영화 추천 함수
def get_movie_recommendation(user_input):
    # 사용자의 감정 분석
    user_sentiment = sentiment_analysis_model(user_input.lower())[0]["label"]
    print(f"감정 분석 결과: {user_sentiment}")

    # 사용자의 감정에 따른 영화 추천
    recommended_movies = []
    for movie, info in movies_data.items():
        if info["genre"] == user_sentiment:
            recommended_movies.append(movie)

    return recommended_movies

# 사용자의 감정을 입력받아서 영화 추천
user_input = input("감정을 입력하세요: ")
recommended_movies = get_movie_recommendation(user_input)

if recommended_movies:
    print(f"추천하는 영화들 (감정: {get_movie_recommendation.user_sentiment}):")
    for movie in recommended_movies:
        print(movie)
else:
    print("해당하는 감정에 대한 영화를 찾을 수 없습니다.")


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


감정 분석 결과: NEGATIVE
해당하는 감정에 대한 영화를 찾을 수 없습니다.
